In [ ]:
import os
import gc

import tqdm
import cv2
import numpy as np
import torch.nn as nn
import torch
from torchvision.transforms import functional as F
from torchvision.transforms import InterpolationMode
from torchvision import transforms
from torchvision.transforms import v2 as transforms_v2 

from pytorch_grad_cam import GradCAM, GradCAMPlusPlus
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image

import matplotlib.pyplot as plt

In [ ]:
os.chdir("..")

In [ ]:
from src.model.classifier import MODELS, get_pretrained_model
from src.dataset.dataset import LycaenidaeDatasetCls
from src.dataset.transform import get_cls_pretrained_transform
from src.utils import get_best_model_weights

In [ ]:
def plot_gradcam(
    dataset: LycaenidaeDatasetCls,
    model: nn.Module,
    target_layers: list[nn.Module],
    resize: tuple[int, int],
    img_idx: int,
    save_dir: str,
    interpolation: int = InterpolationMode.BILINEAR,
    reshape_transform: callable or None = None
):
    os.makedirs(save_dir, exist_ok=True)

    img, label, img_path = dataset._getitem(img_idx)
    filename = img_path.split(os.sep)[-1]
    input_tensor, _ = dataset.__getitem__(img_idx)
    
    input_tensor = torch.unsqueeze(input_tensor, 0)

    preproc_img = torch.as_tensor(img, dtype=torch.float32) 
    preproc_img = preproc_img.permute((2, 0, 1)).contiguous()
    preproc_img = F.resize(preproc_img, resize, interpolation=interpolation).permute(1, 2, 0).numpy().astype(np.uint8) / 255


    targets = [ClassifierOutputTarget(label)]

    with GradCAM(model=model, target_layers=target_layers, reshape_transform=reshape_transform) as cam:
        grayscale_cams = cam(input_tensor=input_tensor, targets=targets)
        cam_image = show_cam_on_image(preproc_img, grayscale_cams[0, :], use_rgb=True)
        
    with GradCAMPlusPlus(model=model, target_layers=target_layers, reshape_transform=reshape_transform) as campp:
        grayscale_campps = campp(input_tensor=input_tensor, targets=targets)
        campp_image = show_cam_on_image(preproc_img, grayscale_campps[0, :], use_rgb=True)

    cam = np.uint8( 255 * grayscale_cams[0, :])
    campp = np.uint8( 255 * grayscale_campps[0, :])
    images = {
        "cam": (np.uint8(255 * preproc_img), cv2.merge([cam, cam, cam]) , cam_image),
        "campp": (np.uint8(255 * preproc_img), cv2.merge([campp, campp, campp]) , campp_image),
    }

    fig, axes = plt.subplots(nrows=2, ncols=3, figsize= (24, 12))
    
    for idx, cam_name in enumerate(images.keys()):
        for ax, img_to_plot in zip(axes[idx], images[cam_name]):
            ax.imshow(img_to_plot)
            ax.axis("off")

    class_name = dataset.class_labels_inv[label]
    axes[0][0].set_title(f"{class_name}. {filename}")
    axes[0][1].set_title("GradCAM, GradCAM++")
    axes[0][2].set_title("Image & GradCAM, GradCAM++")
    plt.tight_layout()
    plt.savefig(os.path.join(save_dir, f"gradcam_{class_name}_{filename.split('.')[0]}.jpg"))
    plt.close()

In [ ]:
METADATA_PATH = "./data/meta_all_groups_v3.csv"
GRADCAM_DIR = "./gradcam_output"

train_size = 0.8
test_size = 0.1
val_size = 0.1
min_images_per_class = 5
seed = 42
device = "cuda:0"
os.makedirs(GRADCAM_DIR, exist_ok=True)

#### Resnet50

In [ ]:
for view in ("top", "bottom"):
    model_name = "resnet50"
    weights_dir = f"./weights_cls/ckpt_all_groups_cbalanced_sampler_upd_26-07-2025_view-{view}_{model_name}"
    gradcam_dir = os.path.join(GRADCAM_DIR, model_name, view)
    resnet50_transforms_no_crop = transforms.Compose(
        [
        transforms_v2.Resize((232, 232)),    
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ]
    )
    test_set = LycaenidaeDatasetCls(
        metadata_path=METADATA_PATH,
        view=view,
        subset="test",
        train_size=0.8,
        test_size=0.1,
        val_size=0.1,
        transform=None,
        model_transform=resnet50_transforms_no_crop,
        min_images_per_class=min_images_per_class,
        device=device,
        seed=seed
    )
    resnet50 = get_pretrained_model(name=model_name, n_classes=test_set.n_classes)
    best_weights_path = get_best_model_weights(dirname=weights_dir, score_name="inv_loss")
    print(f"Best weights: {best_weights_path}")

    resnet50.load_state_dict(torch.load(best_weights_path))
    resnet50 = resnet50.to(device)
    resnet50 = resnet50.eval()

    for idx in tqdm.tqdm(range(test_set.__len__())):
        plot_gradcam(
            test_set,
            resnet50,
            [resnet50.layer1, resnet50.layer2, resnet50.layer3, resnet50.layer4],
            (232, 232),
            idx,
            gradcam_dir
        )
    with open(os.path.join(gradcam_dir, "info.txt"), "w") as f:
        f.write(f"\nModel weights: {best_weights_path}")
        f.write("\nSelected layers: resnet50_bot.layer1, resnet50_bot.layer2, resnet50_bot.layer3, resnet50_bot.layer4")

In [ ]:
del resnet50
gc.collect()
torch.cuda.empty_cache()

#### MobileNetV3Large

In [ ]:
for view in ("top", "bottom"):
    model_name = "mobilenet_v3_large"
    weights_dir = f"./weights_cls/ckpt_all_groups_cbalanced_sampler_upd_26-07-2025_view-{view}_{model_name}"
    gradcam_dir = os.path.join(GRADCAM_DIR, model_name, view)
    mmbnet_transforms_no_crop = transforms.Compose(
        [
        transforms_v2.Resize((232, 232)),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ]
    )
    test_set = LycaenidaeDatasetCls(
        metadata_path=METADATA_PATH,
        view=view,
        subset="test",
        train_size=0.8,
        test_size=0.1,
        val_size=0.1,
        transform=None,
        model_transform=mmbnet_transforms_no_crop,
        min_images_per_class=min_images_per_class,
        device=device,
        seed=seed
    )
    mbnet = get_pretrained_model(name=model_name, n_classes=test_set.n_classes)
    best_weights_path = get_best_model_weights(dirname=weights_dir, score_name="inv_loss")
    print(f"Best weights: {best_weights_path}")

    mbnet.load_state_dict(torch.load(best_weights_path))
    mbnet = mbnet.to(device)
    mbnet = mbnet.eval()

    for idx in tqdm.tqdm(range(test_set.__len__())):
        plot_gradcam(
            test_set,
            mbnet,
            [mbnet.features[0], mbnet.features[5], mbnet.features[10], mbnet.features[15]],
            (232, 232),
            idx,
            gradcam_dir
        )
    with open(os.path.join(gradcam_dir, "info.txt"), "w") as f:
        f.write(f"\nModel weights: {best_weights_path}")
        f.write("\nSelected layers: mbnet.features[0], mbnet.features[5], mbnet.features[10], mbnet.features[15]")

In [ ]:
del mbnet
gc.collect()
torch.cuda.empty_cache()

#### EfficientNetV2L

In [ ]:
for view in ("top", "bottom"):
    model_name = "efficientnet_v2_l"
    weights_dir = f"./weights_cls/ckpt_all_groups_cbalanced_sampler_upd_26-07-2025_view-{view}_{model_name}"
    gradcam_dir = os.path.join(GRADCAM_DIR, model_name, view)
    effnetv2l_transforms_no_crop = transforms.Compose(
        [
        transforms_v2.Resize((480, 480)),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
        ]
    )
    test_set = LycaenidaeDatasetCls(
        metadata_path=METADATA_PATH,
        view=view,
        subset="test",
        train_size=0.8,
        test_size=0.1,
        val_size=0.1,
        transform=None,
        model_transform=effnetv2l_transforms_no_crop,
        min_images_per_class=min_images_per_class,
        device=device,
        seed=seed
    )
    effnet = get_pretrained_model(name=model_name, n_classes=test_set.n_classes)
    best_weights_path = get_best_model_weights(dirname=weights_dir, score_name="inv_loss")
    print(f"Best weights: {best_weights_path}")

    effnet.load_state_dict(torch.load(best_weights_path))
    effnet = effnet.to(device)
    effnet = effnet.eval()

    for idx in tqdm.tqdm(range(test_set.__len__())):
        plot_gradcam(
            test_set,
            effnet,
            [effnet.features[6], effnet.features[7], effnet.features[8]],
            (480, 480),
            idx,
            gradcam_dir
        )
    with open(os.path.join(gradcam_dir, "info.txt"), "w") as f:
        f.write(f"\nModel weights: {best_weights_path}")
        f.write("\nSelected layers: mbnet.features[0], mbnet.features[5], mbnet.features[10], mbnet.features[15]")

In [ ]:
del effnet
gc.collect()
torch.cuda.empty_cache()